# Data Wrangling Template

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import matplotlib.pyplot as plt

## Gather

WeRateDogs Twitter archive

- WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).

In [2]:
#Import csv file
tae = pd.read_csv('twitter-archive-enhanced.csv')

In [4]:
#Create directory 
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [5]:
#Download file from url
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)
response
with open(os.path.join(folder_name, url.split('/')[-1]), mode ='wb') as file:
    file.write(response.content)

In [6]:
#Import tsv file
ip = pd.read_csv('image_predictions/image-predictions.tsv', sep='\t')

Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. 


Image Predictions File

- The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).

- tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
- p1 is the algorithm's #1 prediction for the image in the tweet → golden retriever
- p1_conf is how confident the algorithm is in its #1 prediction → 95%
- p1_dog is whether or not the #1 prediction is a breed of dog → TRUE
- p2 is the algorithm's second most likely prediction → Labrador retriever
- p2_conf is how confident the algorithm is in its #2 prediction → 1%
- p2_dog is whether or not the #2 prediction is a breed of dog → TRUE



## Assess

### Assessment of Twitter Archive Enhanced

In [15]:
tae.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

In [17]:
tae.sample(5)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
1224  714214115368108032                    NaN                  NaN   
535   807059379405148160                    NaN                  NaN   
1714  680440374763077632                    NaN                  NaN   
1946  673705679337693185                    NaN                  NaN   
1767  678396796259975168                    NaN                  NaN   

                      timestamp  \
1224  2016-03-27 22:14:49 +0000   
535   2016-12-09 03:08:45 +0000   
1714  2015-12-25 17:30:01 +0000   
1946  2015-12-07 03:28:45 +0000   
1767  2015-12-20 02:09:34 +0000   

                                                 source  \
1224  <a href="http://twitter.com/download/iphone" r...   
535   <a href="http://twitter.com/download/iphone" r...   
1714  <a href="http://twitter.com/download/iphone" r...   
1946  <a href="http://twitter.com/download/iphone" r...   
1767  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
1224  Happy Easter from the squad! 🐇🐶 13/10 for all ...                  NaN   
535   RT @dog_rates: This is Cali. She arrived preas...         7.829691e+17   
1714  Merry Christmas. My gift to you is this tiny u...                  NaN   
1946  This is Dot. He found out you only pretended t...                  NaN   
1767  These little fellas have opposite facial expre...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
1224                       NaN                        NaN   
535               4.196984e+09  2016-10-03 15:42:44 +0000   
1714                       NaN                        NaN   
1946                       NaN                        NaN   
1767                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
1224  https://twitter.com/dog_rates/status/714214115...                13   
535   https://twitter.com/dog_rates/status/782969140...                12   
1714  https://twitter.com/dog_rates/status/680440374...                11   
1946  https://twitter.com/dog_rates/status/673705679...                 8   
1767  https://twitter.com/dog_rates/status/678396796...                12   

      rating_denominator  name doggo floofer pupper puppo  
1224                  10  None  None    None   None  None  
535                   10  Cali  None    None   None  None  
1714                  10  None  None    None   None  None  
1946                  10   Dot  None    None   None  None  
1767                  10  None  None    None   None  None

In [24]:
tae.describe()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
count  2.356000e+03           7.800000e+01         7.800000e+01   
mean   7.427716e+17           7.455079e+17         2.014171e+16   
std    6.856705e+16           7.582492e+16         1.252797e+17   
min    6.660209e+17           6.658147e+17         1.185634e+07   
25%    6.783989e+17           6.757419e+17         3.086374e+08   
50%    7.196279e+17           7.038708e+17         4.196984e+09   
75%    7.993373e+17           8.257804e+17         4.196984e+09   
max    8.924206e+17           8.862664e+17         8.405479e+17   

       retweeted_status_id  retweeted_status_user_id  rating_numerator  \
count         1.810000e+02              1.810000e+02       2356.000000   
mean          7.720400e+17              1.241698e+16         13.126486   
std           6.236928e+16              9.599254e+16         45.876648   
min           6.661041e+17              7.832140e+05          0.000000   
25%           7.186315e+17              4.196984e+09         10.000000   
50%           7.804657e+17              4.196984e+09         11.000000   
75%           8.203146e+17              4.196984e+09         12.000000   
max           8.874740e+17              7.874618e+17       1776.000000   

       rating_denominator  
count         2356.000000  
mean            10.455433  
std              6.745237  
min              0.000000  
25%             10.000000  
50%             10.000000  
75%             10.000000  
max            170.000000

In [25]:
tae.info()

# tweet_id string?
# timestamp time
# source change text
# text analysis
# retweeted timestamp time
# name analysis
# doggo, floofer, pupper, puppo --> one column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [33]:
# access rows
tae.iloc[1224:1225]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
1224  714214115368108032                    NaN                  NaN   

                      timestamp  \
1224  2016-03-27 22:14:49 +0000   

                                                 source  \
1224  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
1224  Happy Easter from the squad! 🐇🐶 13/10 for all ...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
1224                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
1224  https://twitter.com/dog_rates/status/714214115...                13   

      rating_denominator  name doggo floofer pupper puppo  
1224                  10  None  None    None   None  None

In [36]:
# access columns
tae[['tweet_id', 'timestamp']]

tweet_id                  timestamp
0     892420643555336193  2017-08-01 16:23:56 +0000
1     892177421306343426  2017-08-01 00:17:27 +0000
2     891815181378084864  2017-07-31 00:18:03 +0000
3     891689557279858688  2017-07-30 15:58:51 +0000
4     891327558926688256  2017-07-29 16:00:24 +0000
...                  ...                        ...
2351  666049248165822465  2015-11-16 00:24:50 +0000
2352  666044226329800704  2015-11-16 00:04:52 +0000
2353  666033412701032449  2015-11-15 23:21:54 +0000
2354  666029285002620928  2015-11-15 23:05:30 +0000
2355  666020888022790149  2015-11-15 22:32:08 +0000

[2356 rows x 2 columns]

In [35]:
# build subset of table
tae[['tweet_id', 'timestamp']].iloc[1224:1228]

tweet_id                  timestamp
1224  714214115368108032  2016-03-27 22:14:49 +0000
1225  714141408463036416  2016-03-27 17:25:54 +0000
1226  713919462244790272  2016-03-27 02:43:58 +0000
1227  713909862279876608  2016-03-27 02:05:49 +0000

In [41]:
# build new dataframe
data = tae[['tweet_id', 'timestamp']].iloc[1224:1228]

In [40]:
# query dataframe
data.query('tweet_id == 714214115368108032')

tweet_id                  timestamp
1224  714214115368108032  2016-03-27 22:14:49 +0000

In [45]:
columns = tae.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [ ]:
for column in columns:
    # count number of strings
    # print longer columns with more than 10 strings
    

### Assessment of Image Predictions

### Issues Twitter Archive Enhanced

#### Tidiness

#### Quality


### Issues Image Predictions

#### Tidiness

#### Quality


# Clean


## Clean Twitter Archive Enhanced

#### Define

#### Code

#### Test

# Store, Analyze, and Visualize